# Benchmark Computer Vision (CV) models using Amazon SageMaker Multi-model endpoints (MME) with GPU support

Amazon SageMaker multi-model endpoints with GPU works using NVIDIA Triton Inference Server. NVIDIA Triton Inference Server is open-source inference serving software that simplifies the inference serving process and provides high inference performance. Triton supports all major training and inference frameworks, such as TensorFlow, NVIDIA TensorRT, PyTorch, MXNet, Python, ONNX, XGBoost, scikit-learn, RandomForest, OpenVINO, custom C++, and more. It offers dynamic batching, concurrent execution, post-training quantization, optimal model configuration to achieve high performance inference.

In this notebook, we are going to run benchmark testing for the most popluar CV models using MME on GPU. We will evaluate model performance such as cost per transaction (CPT), inference latency, throughput, and optimum model count per instance on a variety of GPU instances. we will also compile these models using NVIDA TensorRT to demonstrate the performance gain from graph optimization.

This notebook is test on `Pytorch 1.8 Python 3.6 GPU Optimized` kernel

**Here is a list of model we have tested, you can use this notebook to benchmark your own models:**

| Model Name      | Model Repo URL               |
| -----------     | -----------                  |
| vgg16           | pytorch/vision:v0.10.0       |
| alexnet         | pytorch/vision:v0.10.0       |
| resnet50        | pytorch/vision:v0.10.0       |
| yolov5x         | ultralytics/yolov5           |
| 

## Contents
1. [Set up the environment](#Set-up-the-environment)
1. [Build a custom TensorRT (TRT) container](#Build-a-custom-TensorRT-(TRT)-container)
1. [Generate Pretrained Models](#Generate-Pretrained-Models)
1. [Packaging Pytorch model for Triton sever on SageMaker](#Packaging-Pytorch-model-for-Triton-sever-on-SageMaker)
1. [Packaging TensorRT model for Triton sever on SageMaker](#Packaging-TensorRT-model-for-Triton-sever-on-SageMaker)
1. [Create SageMaker MME](#Create-SageMaker-MME)
1. [Run Inference to test SageMaker MME](#Run-Inference-to-test-SageMaker-MME)
1. [Run benchmark testing]()
1. [Analyze benchmark results]()
1. [Clean up]()

## Set up the environment

Installs the dependencies required to package the model and run inferences using Triton server.

Also define the IAM role that will give SageMaker access to the model artifacts and the NVIDIA Triton ECR image.

In [1]:
import sagemaker
from sagemaker import get_execution_role
import torch
import torchvision.models as models
from pathlib import Path

import boto3
import json
import time

role = get_execution_role()
sess = sagemaker.Session()

account = sess.account_id()
bucket = sess.default_bucket() # or use your own custom bucket name
prefix = 'mme-cv-benchmark'

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")

model_name = 'alexnet' #' #change the model name to benchmark different CV models
instance_type = 'ml.g4dn.4xlarge'

OSError: /home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/lib/../../nvidia/cublas/lib/libcublas.so.11: symbol cublasLtHSHMatmulAlgoInit, version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference

Account Id Mapping for triton inference containers

In [292]:
account_id_map = {
    'us-east-1': '785573368785',
    'us-east-2': '007439368137',
    'us-west-1': '710691900526',
    'us-west-2': '301217895009',
    'eu-west-1': '802834080501',
    'eu-west-2': '205493899709',
    'eu-west-3': '254080097072',
    'eu-north-1': '601324751636',
    'eu-south-1': '966458181534',
    'eu-central-1': '746233611703',
    'ap-east-1': '110948597952',
    'ap-south-1': '763008648453',
    'ap-northeast-1': '941853720454',
    'ap-northeast-2': '151534178276',
    'ap-southeast-1': '324986816169',
    'ap-southeast-2': '355873309152',
    'cn-northwest-1': '474822919863',
    'cn-north-1': '472730292857',
    'sa-east-1': '756306329178',
    'ca-central-1': '464438896020',
    'me-south-1': '836785723513',
    'af-south-1': '774647643957'
}

In [293]:
region = boto3.Session().region_name
if region not in account_id_map.keys():
    raise("UNSUPPORTED REGION")

## Generate Pretrained Models

We are going to use the following SageMaker Processing script to generate our pretrained model. This script does the following:

1. download the pretrained model from torch hub

2. jit trace and save the torchscript file

3. output of the processing job is a pytorch model (model.pt) and TesorRT model (model.plan) stored in your desinated S3 output location.

In [294]:
if model_name == "resnet50":
    model = models.resnet50(pretrained=True)
    
elif model_name == "vgg16":
    model = models.vgg16(pretrained=True)    
    
elif model_name == "alexnet":
    model = models.alexnet(pretrained=True)
    
elif model_name == "efficientnet_b7":
    model = models.efficientnet_b7(pretrained=True)
    
elif model_name == "vit-base-patch16-224-in21k":
    model = transformers.ViTModel.from_pretrained(f"google/{model_name}", torchscript=True)
else:
    raise ("THIS MODEL HAS NOT BEEN TESTED")
    
example_input = torch.randn(1, 3, 224, 224)
model.eval()

traced_script_module = torch.jit.trace(model, example_input)

## Packaging Pytorch model for Triton sever on SageMaker

**Note**: SageMaker expects the model tarball file to have a top level directory with the same name as the model defined in the `config.pbtxt`.

```
model_name
├── 1
│   └── model.pt
└── config.pbtxt
```

In [295]:
pytorch_model_path = f"{model_name}-pt/{model_name}/1"

!mkdir -p $pytorch_model_path

traced_script_module.save(f"{pytorch_model_path}/model.pt")

In [296]:
%%writefile {model_name}-pt/{model_name}/config.pbtxt
platform: "pytorch_libtorch"
max_batch_size: 32
input {
  name: "INPUT__0"
  data_type: TYPE_FP32
  dims: [3,224, 224]
}
output {
  name: "OUTPUT__0"
  data_type: TYPE_FP32
  dims: 1000
    
}
instance_group {
  count: 1
  kind: KIND_GPU
}
dynamic_batching {
  preferred_batch_size: 16
}

Overwriting alexnet-pt/alexnet/config.pbtxt


## Generate a TensorRT model

1. Export the pre-trained into an ONNX file. ONNX is intemedia file format which is framework agnostic. It works with models in TF, PyTorch and more.

2. Spin up a Sagemaker processing job to convert ONNX model to a TensorRT model plan. You will export the weights of your model from the framework and load them into your tensorRT network.

### <span style="color:red">For TensorRT models, The hosting instance type must match the instance type the model is compiled on. </span>

In [297]:
onnx_model_path = "onnx-models"
!mkdir -p {onnx_model_path}

torch.onnx.export(
    model,
    example_input,
    f"{onnx_model_path}/{model_name}.onnx",
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=["INPUT__0"],
    output_names=["OUTPUT__0"],
    dynamic_axes={"INPUT__0": {0: "batch_size"}, "OUTPUT__0": {0: "batch_size"}},
)

In [298]:
%%writefile trt_compile.sh
#!/bin/bash
mkdir /opt/ml/processing/output/
/usr/src/tensorrt/bin/trtexec $@

Overwriting trt_compile.sh


In [299]:
base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
mme_triton_image_uri = f"{account_id_map[region]}.dkr.ecr.{region}.{base}" + \
            "/sagemaker-tritonserver:22.07-py3"
print(mme_triton_image_uri)

301217895009.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tritonserver:22.07-py3


In [300]:
import boto3
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput, Processor
from sagemaker import get_execution_role

import uuid

region = boto3.session.Session().region_name

s3_output = f's3://{bucket}/{prefix}/{model_name}/outputs/'


compilation_job = ScriptProcessor(
    image_uri=mme_triton_image_uri,
    command=["/bin/bash"],
    instance_type=instance_type,
    volume_size_in_gb=100,
    instance_count=1,
    base_job_name=f"{prefix}-{model_name.replace('_', '')}",
    role=role,  
)

compilation_job.run(
    code = "trt_compile.sh",
    inputs=[ProcessingInput(
            source=f"{onnx_model_path}/{model_name}.onnx",
            destination='/opt/ml/processing/input_data/'),
           ],
    outputs=[ProcessingOutput(source="/opt/ml/processing/output/", 
                         destination=s3_output)],
    arguments=[f"--onnx=/opt/ml/processing/input_data/{model_name}.onnx", 
               "--saveEngine=/opt/ml/processing/output/model.plan", 
               "--explicitBatch", 
               "--minShapes=INPUT__0:1x3x224x224",
               "--optShapes=INPUT__0:128x3x224x224",
               "--maxShapes=INPUT__0:128x3x224x224",
               "--int8",
               "--verbose"
              ],
        wait=False
)


Job Name:  mme-cv-benchmark-alexnet-2022-12-01-15-57-18-294
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-376678947624/mme-cv-benchmark-alexnet-2022-12-01-15-57-18-294/input/input-1/alexnet.onnx', 'LocalPath': '/opt/ml/processing/input_data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-376678947624/mme-cv-benchmark-alexnet-2022-12-01-15-57-18-294/input/code/trt_compile.sh', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-376678947624/mme-cv-benchmark/alexnet/outputs/', 'LocalPath': '/opt/ml/processing/output/', 'S3UploadMode': 'EndOfJ

In [301]:
job_waiter = sm_client.get_waiter('processing_job_completed_or_stopped')
print("Waiting for job to complete")
job_waiter.wait(ProcessingJobName=compilation_job.latest_job.job_name)
print("Job completed")

Waiting for job to complete
Job completed


## Packaging TensorRT model for Triton sever on SageMaker

**Note**: SageMaker expects the model tarball file to have a top level directory with the same name as the model defined in the `config.pbtxt`.


```
model_name
├── 1
│   └── model.plan
└── config.pbtxt
```

In [302]:
!mkdir -p {model_name}-trt/{model_name}/1/
!aws s3 cp {s3_output}model.plan {model_name}-trt/{model_name}/1/

download: s3://sagemaker-us-west-2-376678947624/mme-cv-benchmark/alexnet/outputs/model.plan to alexnet-trt/alexnet/1/model.plan


In [303]:
%%writefile {model_name}-trt/{model_name}/config.pbtxt
platform: "tensorrt_plan"
max_batch_size: 32
input {
  name: "INPUT__0"
  data_type: TYPE_FP32
  dims: [3,224, 224]
}
output {
  name: "OUTPUT__0"
  data_type: TYPE_FP32
  dims: 1000
}
instance_group {
  count: 1
  kind: KIND_GPU
}
dynamic_batching {
  preferred_batch_size: 16
}

Overwriting alexnet-trt/alexnet/config.pbtxt


### load local triton sever

In [304]:
import os

cur_dir = os.getcwd()
print(f"docker run --gpus=all --rm -p8000:8000 -p8001:8001 -p8002:8002 -v{cur_dir}/{model_name}-trt:/models nvcr.io/nvidia/tritonserver:22.07-py3 tritonserver --model-repository=/models")

docker run --gpus=all --rm -p8000:8000 -p8001:8001 -p8002:8002 -v/home/ec2-user/SageMaker/gpu-test/alexnet-trt:/models nvcr.io/nvidia/tritonserver:22.07-py3 tritonserver --model-repository=/models


In [97]:
!nvidia-smi

Tue Nov 22 17:34:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    26W /  70W |    773MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [98]:
!curl localhost:8002/metrics

# HELP nv_inference_request_success Number of successful inference requests, all batch sizes
# TYPE nv_inference_request_success counter
nv_inference_request_success{model="efficientnet_b7",version="1"} 0.000000
# HELP nv_inference_request_failure Number of failed inference requests, all batch sizes
# TYPE nv_inference_request_failure counter
nv_inference_request_failure{model="efficientnet_b7",version="1"} 0.000000
# HELP nv_inference_count Number of inferences performed (does not include cached requests)
# TYPE nv_inference_count counter
nv_inference_count{model="efficientnet_b7",version="1"} 0.000000
# HELP nv_inference_exec_count Number of model executions performed (does not include cached requests)
# TYPE nv_inference_exec_count counter
nv_inference_exec_count{model="efficientnet_b7",version="1"} 0.000000
# HELP nv_inference_request_duration_us Cumulative inference request duration in microseconds (includes cached requests)
# TYPE nv_inference_request_duration_us counter
nv_infer

In [178]:
import numpy as np
from torchvision import transforms
from PIL import Image
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype

# preprocessing function
def rn50_preprocess(img_path="img1.jpg"):
    img = Image.open(img_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(img).numpy()

transformed_img = np.array([rn50_preprocess(img_path="dog.jpg")])

In [180]:
client = httpclient.InferenceServerClient(url="localhost:8000")

inputs = httpclient.InferInput("INPUT__0", [1,3,224,224], datatype="FP32")
inputs.set_data_from_numpy(transformed_img, binary_data=True)

outputs = httpclient.InferRequestedOutput("OUTPUT__0", binary_data=True, class_count=1000)

# Querying the server

for x in range(1000):
    results = client.infer(model_name=model_name, inputs=[inputs], outputs=[outputs])
    inference_output = results.as_numpy('OUTPUT__0')
    print(inference_output[0][:5])

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

...

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14

[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:146107' b'1.174235:146875'
 b'1.168589:14779']
[b'1.247162:135355' b'1.213870:9403' b'1.200508:14